identified via org tag.

monthly stats (org id, name, count of txns, sum of transaction amount)

In [29]:
import sys
from datetime import datetime

sys.path.insert(1, '../../scripts/')
from s3_support import *

In [2]:
orgs = pd.read_csv("org_download-2.csv")

In [10]:
len_acceptiva = len(orgs[orgs['Tags'].str.contains('accept', case=False).fillna(False)])

print("{:,} entries".format(len(orgs)))
print("{:,} acceptiva entries".format(len_acceptiva))

15,093 entries
48 acceptiva entries


In [11]:
acceptiva_orgs = orgs[orgs['Tags'].str.contains('accept', case=False).fillna(False)]

acceptiva_orgs_list = acceptiva_orgs['Id'].tolist()

In [19]:
q = """select 
            org,
            date_trunc('month', date) as month,
            count(distinct(id)) as trans_count,
            sum(amount) as trans_vol
        from transactions
        where
            status='A' and
            date>=dateadd(month,-2,current_date) and
            org in ({})
        group by org, date_trunc('month', date)"""
q = q.format(", ".join([str(i) for i in acceptiva_orgs_list]))

acceptiva = redshift_query_read(q, schema='production')

In [20]:
acceptiva.head()

,org,month,trans_count,trans_vol
0,446202,2024-09-01,34,29455.91
1,446202,2024-11-01,71,18046.85
2,448218,2024-10-01,44,5990.00
3,450988,2024-10-01,9,3690.50
4,445438,2024-09-01,29,1433.70


In [25]:
w_name = acceptiva.merge(orgs[['Id', 'Org Name']], left_on='org', right_on='Id', how='left')

len(acceptiva), len(w_name)

(80, 80)

In [27]:
w_name['name'] = w_name['Org Name']
w_name.drop(['Id', 'Org Name'], axis=1, inplace=True)
w_name.tail()

,org,month,trans_count,trans_vol,name
75,446202,2024-10-01,96,31032.01,There With Care
76,445438,2024-11-01,49,4099.00,Teens for Christ
77,449159,2024-11-01,1554,119703.52,Christian Health Service Corps
78,446710,2024-11-01,63,2705.08,Brothers on a Road Less Traveled
79,446124,2024-11-01,4,2053.00,The Catholic Telegraph


In [34]:
# limit to last full month
last_month = datetime.now().month - 1

acc_last_month = w_name[w_name['month'].dt.month==last_month]

In [35]:
len(acc_last_month)

29

In [37]:
year = datetime.now().year

acc_last_month.to_csv("acceptiva.{}-{}.csv".format(year, last_month), index=False)

# GT 2023

In [41]:
q = '''select
            count(distinct(org)) as orgs,
            sum(case when recurring=0 or recurring_origin=1 then amount else null end) as total_amount,
            count(distinct(case when recurring=0 or recurring_origin=1 then id else null end)) as trans_count,
            max(amount) as max_amount,
            avg(case when recurring=0 or recurring_origin=1 then amount else null end) as avg_amount,
            count(distinct(case when recurring_origin=1 then id else null end)) as recurring_count,
            count(distinct(case when (source='PP' or channel !=0) then id else null end)) as digital_wallet_count
        from transactions
        where
            (status='A' or status='P') and
            org in ({}) and
            (date='2023-11-28' or 
                (date='2023-11-29' and hour<5))'''.format(', '.join([str(o) for o in acceptiva_orgs_list]))
redshift_query_read(q, schema='production')

,orgs,total_amount,trans_count,max_amount,avg_amount,recurring_count,digital_wallet_count
0,20,111752.77,644,5175.0,173.529146,26,10
